# Exploration

A notebook used to explore FRED data

## Setup    

In [6]:
# Load environment variables using dotenv

from dotenv import load_dotenv

load_dotenv()

True

In [7]:
# Setup autoreload of modules
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from update_path import ensure_src_in_path

ensure_src_in_path()

/Users/coryhuebner/workspace/fed/fred-data/src already in path


In [9]:
# Get the FRED API client

from fred_data import api

fred_client = api.FredApiClient()

In [10]:
## Run everything above this cell for setup

## Data Exploration

### Releases

In [11]:
# Fetch all current releases
from fred_data.releases import get_releases

releases = get_releases(fred_client)
releases_df = releases.releases.sort("name")

# Get an idea of the amount/types of data stored in releases
print(f"Release summary")
print(f"Count: {releases.count}")
print(f"Realtime range: {releases.realtime_start} to {releases.realtime_end}, Duration: {releases.realtime_end - releases.realtime_start}")
releases.releases.describe()

Release summary
Count: 315
Realtime range: 2025-03-12 to 2025-03-12, Duration: 0:00:00


statistic,id,realtime_start,realtime_end,name,press_release,link,notes
str,f64,str,str,str,f64,str,str
"""count""",315.0,"""315""","""315""","""315""",315.0,"""276""","""123"""
"""null_count""",0.0,"""0""","""0""","""0""",0.0,"""39""","""192"""
"""mean""",288.031746,"""2025-03-12 00:00:00""","""2025-03-12 00:00:00""",null,0.542857,null,null
"""std""",152.199623,null,null,null,null,null,null
"""min""",9.0,"""2025-03-12""","""2025-03-12""","""A Millennium of Macroeconomic …",0.0,"""http://data.imf.org/?sk=3E40CD…","""AMERIBOR® (American Interbank …"
"""25%""",172.0,"""2025-03-12""","""2025-03-12""",null,null,null,null
"""50%""",301.0,"""2025-03-12""","""2025-03-12""",null,null,null,null
"""75%""",406.0,"""2025-03-12""","""2025-03-12""",null,null,null,null
"""max""",769.0,"""2025-03-12""","""2025-03-12""","""Zillow Home Value Index (ZHVI)""",1.0,"""https://www2.optimalblue.com/o…","""With the issue dated January 8…"


In [12]:
# Fetch older and current releases

from datetime import date, timedelta
from fred_data.releases import get_releases

archived_releases = get_releases(fred_client, realtime_start=date.today() - timedelta(days=60), realtime_end=date.today())
archived_releases_df = archived_releases.releases

# Get an idea of the amount/types of data stored in releases
print(f"Archived Release summary")
print(f"Count: {archived_releases.count}")
print(f"Realtime range: {archived_releases.realtime_start} to {archived_releases.realtime_end}, Duration: {archived_releases.realtime_end - archived_releases.realtime_start}")
archived_releases.releases.describe()

Archived Release summary
Count: 309
Realtime range: 2025-01-12 to 2025-03-13, Duration: 60 days, 0:00:00


statistic,id,realtime_start,realtime_end,name,press_release,link,notes
str,f64,str,str,str,f64,str,str
"""count""",309.0,"""309""","""309""","""309""",309.0,"""270""","""121"""
"""null_count""",0.0,"""0""","""0""","""0""",0.0,"""39""","""188"""
"""mean""",287.640777,"""2025-01-12 00:00:00""","""2025-03-13 00:00:00""",null,0.540453,null,null
"""std""",152.758267,null,null,null,null,null,null
"""min""",9.0,"""2025-01-12""","""2025-03-13""","""A Millennium of Macroeconomic …",0.0,"""http://data.imf.org/?sk=3E40CD…","""AMERIBOR® (American Interbank …"
"""25%""",170.0,"""2025-01-12""","""2025-03-13""",null,null,null,null
"""50%""",302.0,"""2025-01-12""","""2025-03-13""",null,null,null,null
"""75%""",403.0,"""2025-01-12""","""2025-03-13""",null,null,null,null
"""max""",769.0,"""2025-01-12""","""2025-03-13""","""Zillow Home Value Index (ZHVI)""",1.0,"""https://www2.optimalblue.com/o…","""With the issue dated January 8…"


### Release Dates

In [13]:
# Get release dates for today and tomorrow
from datetime import date, timedelta

from fred_data.releases import get_release_dates

today = date.today()
tomorrow = today + timedelta(days=1)
release_dates = get_release_dates(fred_client, realtime_start=today, realtime_end=tomorrow, include_release_dates_with_no_data=True)
release_dates_df = release_dates.release_dates

print(f"Release Dates Summary")
print(f"Count: {release_dates.count}")
print(f"Realtime range: {release_dates.realtime_start} to {release_dates.realtime_end}, Duration: {release_dates.realtime_end - release_dates.realtime_start}")
release_dates_df.describe()

Release Dates Summary
Count: 69
Realtime range: 2025-03-13 to 2025-03-14, Duration: 1 day, 0:00:00


statistic,release_id,release_name,release_last_updated,date
str,f64,str,str,str
"""count""",69.0,"""69""","""69""","""69"""
"""null_count""",0.0,"""0""","""0""","""0"""
"""mean""",314.753623,null,"""2025-03-09 19:51:36.579710+00:…","""2025-03-13 10:46:57.391000"""
"""std""",168.298353,null,null,null
"""min""",18.0,"""CBOE Market Statistics""","""2025-01-07 20:50:46+00:00""","""2025-03-13"""
"""25%""",190.0,null,"""2025-03-12 20:16:58+00:00""","""2025-03-13"""
"""50%""",304.0,null,"""2025-03-13 11:03:05+00:00""","""2025-03-13"""
"""75%""",455.0,null,"""2025-03-13 13:01:10+00:00""","""2025-03-14"""
"""max""",769.0,"""Z.1 Financial Accounts of the …","""2025-03-13 16:06:05+00:00""","""2025-03-14"""


In [14]:
# Get release dates from the past into the future
from datetime import date, timedelta

from fred_data.releases import get_release_dates

release_dates_over_time = get_release_dates(
    fred_client,
    realtime_start=date.today() - timedelta(days=30),
    realtime_end=date.today() + timedelta(days=10),
    include_release_dates_with_no_data=True
)
release_dates_over_time_df = release_dates_over_time.release_dates

print(f"Release Dates Summary")
print(f"Count: {release_dates_over_time.count}")
print(f"Realtime range: {release_dates_over_time.realtime_start} to {release_dates_over_time.realtime_end}, Duration: {release_dates_over_time.realtime_end - release_dates_over_time.realtime_start}")
release_dates_over_time_df.describe()

Release Dates Summary
Count: 983
Realtime range: 2025-02-11 to 2025-03-23, Duration: 40 days, 0:00:00


statistic,release_id,release_name,release_last_updated,date
str,f64,str,str,str
"""count""",983.0,"""983""","""983""","""983"""
"""null_count""",0.0,"""0""","""0""","""0"""
"""mean""",306.861648,null,"""2025-03-09 12:42:00.620549+00:…","""2025-03-02 03:44:07.812000"""
"""std""",155.231071,null,null,null
"""min""",9.0,"""ADP National Employment Report""","""2023-11-28 21:36:04+00:00""","""2025-02-11"""
"""25%""",198.0,null,"""2025-03-12 15:04:04+00:00""","""2025-02-20"""
"""50%""",326.0,null,"""2025-03-13 08:32:03+00:00""","""2025-03-03"""
"""75%""",445.0,null,"""2025-03-13 12:02:08+00:00""","""2025-03-12"""
"""max""",769.0,"""Zillow Home Value Index (ZHVI)""","""2025-03-13 16:06:05+00:00""","""2025-03-23"""


### Series

A series is a collection of observations over time. A series includes metadata about itself.

Unanswered Question: How do you tie a series (or its observations) back to the source(s) that generated that series or observation?

In [15]:
# Get series search results from the API
import polars as pl

from fred_data.series import get_search_results

search_text = "illinois unemployment"

search_results = get_search_results(
    fred_client,
    search_text=search_text,
)
search_results_df = search_results.series

print(f"Search Results")
print(f"Count: {search_results.count}")
print(f"Realtime range: {search_results.realtime_start} to {search_results.realtime_end}, Duration: {search_results.realtime_end - search_results.realtime_start}")
search_results_df_summary = pl.DataFrame() if search_results_df.is_empty() else search_results_df.describe()
search_results_df_summary

Search Results
Count: 1186
Realtime range: 2025-03-13 to 2025-03-13, Duration: 0:00:00


statistic,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes
str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str
"""count""","""1186""","""1186""","""1186""","""1186""","""1186""","""1186""","""1186""","""1186""","""1186""","""1186""","""1186""","""1186""","""1186""",1186.0,1186.0,"""1144"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,"""42"""
"""mean""",null,"""2025-03-13 00:00:00""","""2025-03-13 00:00:00""",null,"""1991-12-04 16:44:06.880000""","""2023-10-16 01:37:07.993000""",null,null,null,null,null,null,"""2024-08-22 18:46:53.230185+00:…",0.934233,1.515177,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,1.887304,2.997358,null
"""min""","""ALLMARGATIL""","""2025-03-13""","""2025-03-13""","""All Marginally Attached Worker…","""1948-01-01""","""2008-12-01""","""Annual""","""A""","""Dollars""","""$""","""Not Seasonally Adjusted""","""NSA""","""2009-03-12 20:01:08+00:00""",0.0,0.0,"""A state's labor-force particip…"
"""25%""",null,"""2025-03-13""","""2025-03-13""",null,"""1990-01-01""","""2023-01-01""",null,null,null,null,null,null,"""2024-05-01 15:51:12+00:00""",0.0,1.0,null
"""50%""",null,"""2025-03-13""","""2025-03-13""",null,"""1990-01-01""","""2023-10-01""",null,null,null,null,null,null,"""2024-12-12 15:31:53+00:00""",1.0,1.0,null
"""75%""",null,"""2025-03-13""","""2025-03-13""",null,"""1990-01-01""","""2024-12-01""",null,null,null,null,null,null,"""2025-02-05 16:20:02+00:00""",1.0,1.0,null
"""max""","""WIKENO9URN""","""2025-03-13""","""2025-03-13""","""Unemployment Rate in Woodford …","""2020-04-04""","""2025-03-01""","""Weekly, Ending Saturday""","""W""","""Thousands of Dollars""","""Thous. of $""","""Smoothed Seasonally Adjusted""","""SSA""","""2025-03-07 13:37:06+00:00""",31.0,39.0,"""U-6 unemployment rate, as defi…"


In [16]:
# Get all series contained within a given release

from fred_data.series import get_series_for_release


# Grabbed from releases_df from cells above
stl_financial_stress_release_id = 245
stl_financial_stress_multi_series = get_series_for_release(fred_client, stl_financial_stress_release_id)
stl_financial_stress_multi_series_df = stl_financial_stress_multi_series.series

print(f"Release Series Results")
print(f"Count: {stl_financial_stress_multi_series.count}")
print(f"Realtime range: {stl_financial_stress_multi_series.realtime_start} to {stl_financial_stress_multi_series.realtime_end}, Duration: {stl_financial_stress_multi_series.realtime_end - stl_financial_stress_multi_series.realtime_start}")
st_louis_financial_stress_series_summary = pl.DataFrame() if stl_financial_stress_multi_series_df.is_empty() else stl_financial_stress_multi_series_df.describe()
st_louis_financial_stress_series_summary


Release Series Results
Count: 6
Realtime range: 2025-03-13 to 2025-03-13, Duration: 0:00:00


statistic,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes
str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str
"""count""","""6""","""6""","""6""","""6""","""6""","""6""","""6""","""6""","""6""","""6""","""6""","""6""","""6""",6.0,6.0,"""6"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,"""0"""
"""mean""",null,"""2025-03-13 00:00:00""","""2025-03-13 00:00:00""",null,"""1989-07-02 12:00:00""","""2022-07-02 12:00:00""",null,null,null,null,null,null,"""2022-08-03 09:20:02.166666+00:…",20.333333,20.333333,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,19.946595,19.946595,null
"""min""","""RTWEXAFEGS""","""2025-03-13""","""2025-03-13""","""Real Advanced Foreign Economie…","""1973-01-01""","""2019-12-01""","""Monthly""","""M""","""Index Jan 2006=100""","""Index Jan 2006=100""","""Not Seasonally Adjusted""","""NSA""","""2020-01-02 21:21:01+00:00""",3.0,3.0,"""Averages of daily figures. Ser…"
"""25%""",null,"""2025-03-13""","""2025-03-13""",null,"""1973-01-01""","""2019-12-01""",null,null,null,null,null,null,"""2020-01-02 21:21:02+00:00""",6.0,6.0,null
"""50%""",null,"""2025-03-13""","""2025-03-13""",null,"""2006-01-01""","""2025-02-01""",null,null,null,null,null,null,"""2025-03-03 21:19:02+00:00""",21.0,21.0,null
"""75%""",null,"""2025-03-13""","""2025-03-13""",null,"""2006-01-01""","""2025-02-01""",null,null,null,null,null,null,"""2025-03-03 21:19:03+00:00""",28.0,28.0,null
"""max""","""TWEXOPA""","""2025-03-13""","""2025-03-13""","""Real OITP Dollar Index (Goods …","""2006-01-01""","""2025-02-01""","""Monthly""","""M""","""Index Mar 1973=100""","""Index Mar 1973=100""","""Not Seasonally Adjusted""","""NSA""","""2025-03-03 21:19:03+00:00""",56.0,56.0,"""For more information about tra…"


### Observations

Observations are the actual data for a given series. Multiple vintages of observations can exist for a given series to faciliate:
1. New data coming in for a given time period after a series has already been published for that time period for the series
2. Changes in how a series is derived/calculated

[ALFRED keeps track of old vintage data, where-as FRED only shows the newest vintage of data](http://fred.stlouisfed.org/docs/api/fred/fred_vs_alfred.html)

In [5]:
from datetime import date, timedelta

# Just using Plotly express for easy graphing
import plotly.express as px

from fred_data.observations import get_observations_for_series
from fred_data.series import get_series_by_id

# Grabbed from results of "Series" section cells
real_broad_dollar_index_series_id = "RTWEXBGS"
real_broad_dollar_index_series = get_series_by_id(fred_client, series_id=real_broad_dollar_index_series_id)
today = date.today()
real_broad_dollar_index_observations = get_observations_for_series(
    fred_client,
    series_id=real_broad_dollar_index_series_id,
    observation_start=today - timedelta(days=365*30), # 30 years
    observation_end=today,
)
real_broad_dollar_index_observations_df = real_broad_dollar_index_observations.observations

observations_chart = px.line(
    x=real_broad_dollar_index_observations_df["date"],
    y=real_broad_dollar_index_observations_df["value"],
    title=real_broad_dollar_index_series.title,
    labels={
        "x": "Date",
        "y": real_broad_dollar_index_series.units
    }
)
observations_chart